In [21]:
%load_ext autoreload

%autoreload 2

from RNN2DFA.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from RNN2DFA.RNNClassifier import RNNClassifier
from RNN2DFA.Training_Functions import mixed_curriculum_train
import Tomita_Grammars 
from lstar_extraction.Training_Functions import make_test_set,make_train_set_for_target
from RNNexplainer import Explainer
import pandas as pd
import LTL2DFA as ltlf2dfa





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# construct a dfa that implements alternating bit protocol
import LTL2DFA as ltlf2dfa
dfa=ltlf2dfa.DFA()

# alphabet
alphabet="abcd" 
""" 
a = bit 0 
b = bit 1
c = bit 0 acknowledge
d = bit 1 acknowledge
 """
dfa.alphabet = [character for character in alphabet]
dfa.Q = [0,1,2,3,4] # state 4 denotes rejecting state for all unalowed move
dfa.q0 = 0
dfa.F= [0,2] 
dfa.delta = {
    0: {
        '0001': 0,
        '0010': 4,
        '0100': 4,
        '1000': 1,
        '0000': 0 # implements empty input
    },
    1: {
        '0001': 4,
        '0010': 2,
        '0100': 4,
        '1000': 1,
        '0000': 1 # implements empty input
    },
    2: {
        '0001': 4,
        '0010': 2,
        '0100': 3,
        '1000': 4,
        '0000': 2 # implements empty input
    },
    3: {
        '0001': 0,
        '0010': 4,
        '0100': 3,
        '1000': 4,
        '0000': 3 # implements empty input
    },
    4: {
        '0001': 4,
        '0010': 4,
        '0100': 4,
        '1000': 4,
        '0000': 4 # implements empty input
    }
}
target_formula="alternating bit protocol"
generator_dfa=dfa 

In [24]:



# make training set on grammars
target = Tomita_Grammars.tomita_email
alphabet = "pqradmn"
generator_dfa.classify_word=target
target_formula="mail string match"


# # use a dfa to generate training set
# target_formula="G(b->X(~a))"
# alphabet="abc"
# fout=open("output/log.txt", "a")
# fout.write(".........................................................................\n")
# fout.write("Target: "+ target_formula)
# fout.write("\n")
# fout.close()
# import LTL2DFA as ltlf2dfa
# generator_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=target_formula)
# print(generator_dfa)


train_set = make_train_set_for_target(generator_dfa.classify_word,alphabet)

# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


# print(train_set)
print("\n\nPositive examples:")
for key in train_set:
    if(train_set[key]):
        print(key)
# print(train_set)


made train set of size: 366 , of which positive examples: 100


Positive examples:
paqd
qapd
qaqd
papd
ppapd
pqaqd
qnaqd
qqaqd
pnaqd
qaqpd
qapqd
pmaqd
qappd
qmaqd
qqapd
qmapd
qaqqd
qnapd
pappd
pnapd
pmapd
papqd
paqqd
qpaqd
qpapd
ppaqd
paqpd
pqapd
qappqd
qmmapd
qpmapd
paqpqd
qnnaqd
pqnaqd
pmaqpd
qpnapd
pmqapd
qmnaqd
pqpapd
pmaqqd
qnpapd
qppaqd
pmpaqd
qqpaqd
qqnapd
pnappd
pappqd
papppd
pnnapd
qqmaqd
ppqaqd
pnaqpd
pnqapd
pppapd
qnapqd
qapqqd
pmappd
pnapqd
qnpaqd
pmpapd
ppqapd
qqapqd
qmaqqpd
qqmapqd
qppaqpd
pqqpapd
pmpnapd
qmqnaqd
qppmaqd
pmqapqd
qqaqpqd
pmnnapd
qnppapd
pmpapqd
qpnaqpd
pnpqaqd
pnqappd
pappqpd
qpqpaqd
qnapqqd
paqpqqd
pqappqd
qmmapqd
pqqqapd
qppmapd
qqnaqpd
ppmnqapd
qqpnapqd
qpqpaqpd
pnmnaqqd
qpqmnaqd
pmmpqnaqd
ppmnaqqpd
qmmmpnaqd
pnqnnmaqd
pqnnapppd
qpnqnmaqd
ppppqnappd
pnmnqmqqaqpd
pqqnmmqapqqd


In [25]:
# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  
dfa_from_rnn=rnn 
# statistics

def percent(num,digits=2):
    tens = pow(10,digits)
    return int(100*num*tens)/tens

test_set = train_set 
print("testing on train set, i.e. test set is train set")
# we're printing stats on the train set for now, but you can define other test sets by using
# make_train_set_for_target

n = len(test_set)
print("test set size:", n)
pos = len([w for w in test_set if generator_dfa.classify_word(w)])
print("of which positive:",pos,"("+str(percent(pos/n))+")")
rnn_target = len([w for w in test_set if dfa_from_rnn.classify_word(w)==generator_dfa.classify_word(w)])
print("rnn score against target on test set:                             ",rnn_target,"("+str(percent(rnn_target/n))+")")

# dfa_from_rnn=generator_dfa

current average loss is:  0.1753828246865356
current average loss is:  0.07014514845714234
current average loss is:  0.04538774555712416
current average loss is:  0.050422424940686474
current average loss is:  0.020722455132687306
current average loss is:  0.01311825012248722
current average loss is:  0.009999133198639083
current average loss is:  0.007804854971463563
current average loss is:  0.0063430318415490655
current average loss is:  0.005328063430681906
current average loss is:  0.0026285493744357506
current average loss is:  0.0015394853385116314
current average loss is:  0.0009797159053401595
current average loss is:  0.000657816575492912
classification loss on last batch was: 0.0004897981487661845
testing on train set, i.e. test set is train set
test set size: 366
of which positive: 100 (27.32)
rnn score against target on test set:                              266 (72.67)


In [20]:


# use a query LTL formula
query_formula="F(a)"
query_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=query_formula)



# make test_set supported by query dfa
sample_test_set=make_train_set_for_target(query_dfa.classify_word, alphabet)

# only consider sample that are true
test_set=[]
for key in sample_test_set:
    if(sample_test_set[key]):
        test_set.append(key)
# print(test_set[:30])

from PACTeacher.pac_teacher import PACTeacher as Teacher 

# query_dfa=None

if(query_dfa is None):
    query_formula=None
    test_set=make_test_set(alphabet)
    raise SystemError


from RNNexplainer import Traces
traces=Traces(rnn, alphabet)
traces.label_from_network(test_set)
traces.write_in_file()


explainer=Explainer(alphabet=[character for character in alphabet])
teacher = Teacher(dfa_from_rnn,epsilon=.05, delta=.05, max_trace_length=20, max_formula_depth=10, query_dfa=query_dfa)
from time import clock
start_time=clock()
flag=teacher.teach(explainer,traces)
end_time=clock()


print("\n\nepsilon=", teacher.epsilon, "delta=", teacher.delta, "max_trace_length=", teacher.max_trace_length)
print("query:", query_formula)
print("final ltl: ", explainer.ltl)

fout=open("output/log.txt", "a")
fout.write("\n\nquery: "+query_formula)
fout.write("\nfinal LTL: "+ explainer.ltl)
if(not flag):
    fout.write(" [incomplete]")
    print("incomplete formula")
fout.write("\n\n")

print("\nTime taken:", end_time-start_time)

fout.close()


test_set = train_set 

fout=open("output/log.txt", "a")
# fout.write("rnn score against target on test set:                             "+str(rnn_target)+"("+str(percent(rnn_target/n))+")")
# fout.write("\n")

performance_ltl = len([w for w in test_set if dfa_from_rnn.classify_word(w)==explainer.dfa.classify_word(w)])
print("extracted LTL score against rnn on test set:                      ",performance_ltl,"("+str(percent(performance_ltl/n))+")")
performance_ltl_with_target = len([w for w in test_set if explainer.dfa.classify_word(w)==generator_dfa.classify_word(w)])
print("extracted LTL score against target on rnn's test set:             ",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

performance_ltl = len([w for w in test_set if (dfa_from_rnn.classify_word(w)and query_dfa.classify_word(w)) ==explainer.dfa.classify_word(w)])
print("extracted LTL score against rnn on test set (with query):         ",performance_ltl,"("+str(percent(performance_ltl/n))+")")
# fout.write("extracted LTL score against rnn on test set (with query):         "+str(performance_ltl)+"("+str(percent(performance_ltl/n))+")\n")
performance_ltl_with_target = len([w for w in test_set if explainer.dfa.classify_word(w)== (generator_dfa.classify_word(w) and query_dfa.classify_word(w))])
print("extracted LTL score against target on rnn's test set (with query):",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

# fout.write("extracted LTL score against target on rnn's test set (with query):"+str(performance_ltl_with_target)+"("+str(percent(performance_ltl_with_target/n))+")\n")
fout.close()


# report in a pandas file
result = pd.DataFrame(columns=['target', 
                                'query', 
                                'explanation', 
                                'status', 
                                'rnn score', 
                                'explanation score', 
                                'explanation score on ground truth',
                                'extraction time'
                                ])

result = result.append(
    {
        'target':target_formula,
        'query':query_formula,
        'explanation':explainer.ltl,
        'status':flag,
        'rnn score':percent(rnn_target/n),
        'explanation score':percent(performance_ltl/n),
        'explanation score on ground truth':percent(performance_ltl_with_target/n),
        'extraction time': end_time-start_time
    }, ignore_index=True
)
print(result.to_string(index=False))
result.to_csv('output/result.csv', header=False, index=False, mode='a')


made train set of size: 15654 , of which positive examples: 7874


positive traces---> 
['rapd']


negative traces---> 
['a']



start formula depth: 1
learned LTL formula: r
new counterexample: maanqapamqnqd  should be accepted by implementation


positive traces---> 
['rapd', 'maanqapamqnqd']


negative traces---> 
['a']



0  iteration complete



start formula depth: 1
learned LTL formula: (F p)
new counterexample: p  should be rejected by implementation


positive traces---> 
['rapd', 'maanqapamqnqd']


negative traces---> 
['a', 'p']



1  iteration complete



start formula depth: 2
learned LTL formula: (X a)
new counterexample: ma  should be rejected by implementation


positive traces---> 
['rapd', 'maanqapamqnqd']


negative traces---> 
['a', 'p', 'ma']



2  iteration complete



start formula depth: 2
learned LTL formula: (F d)
new counterexample: nmqmramnrpqd  should be rejected by implementation


positive traces---> 
['rapd', 'maanqapamqnqd']


negative traces---> 
['a',

In [9]:
# read result
df=pd.read_csv("output/result.csv",header=None)
df.columns=['target', 
            'query', 
            'explanation', 
            'status', 
            'rnn score', 
            'explanation score', 
            'explanation score on ground truth',
            'extraction time'
            ]
print(df.to_string(index=False))

target                      query                      explanation  status  rnn score  explanation score  explanation score on ground truth  extraction time
 alternating bit protocol  a&X(c)&X(X(b))&X(X(X(d)))                            false    True     100.00              99.76                              99.76         0.096940
 alternating bit protocol                    a->F(c)      (~ (d U ((b | c) | (G a))))   False     100.00              73.98                              73.98       217.447062
 alternating bit protocol  (a->F(c))&(G(~b))&(G(~d))   (((d | a) | (c | b)) -> (X c))   False     100.00              73.26                              73.26        71.138986
        mail string match                       G(p)                            false    True       0.00             100.00                              12.22         0.103343
        mail string match                      ~F(d)            ((~ a) U ((F a) & r))   False       0.00              61.35                